# jfleg/correction

In [10]:
import pandas as pd
from utils import *
from tqdm import tqdm
from transformers import T5TokenizerFast, T5Model, T5Config
from torch.utils.data import DataLoader, ConcatDataset, WeightedRandomSampler, Subset
from Dataset import T5Dataset, JflegDataset
import os
import re
import pandas as pd
from torch.utils.data import Dataset
from transformers import T5TokenizerFast

In [1]:
import spacy

In [2]:
def parsing_(sent):
    doc = nlp(sent)
    docsoup = []

    for token in doc:
        if token.is_punct:
            docsoup.append(str(token))
        else:
            info = [str(token.dep_)]
            info += [str(token.pos_)]
            info += [key[key.find("_") + 1:] for key in nlp.vocab.morphology.tag_map[token.tag_].keys() if
                     type(key) == str]
            docsoup.append("_".join(info))

    parsing = " ".join(docsoup)
    ret = "sentence: " + sent + " parsing: " + parsing
    return ret

In [4]:
parsing_(sent)

'sentence: It has a high density population because its small territory parsing: nsubj_PRON_prs ROOT_AUX_sing_three_pres_fin det_DET amod_ADJ_pos compound_NOUN_sing dobj_NOUN_sing prep_SCONJ poss_DET_yes_prs amod_ADJ_pos pobj_NOUN_sing'

In [3]:
sent = "It has a high density population because its small territory"
nlp = spacy.load("en_core_web_lg")

In [10]:
nlp

In [8]:
"It has a high density population because its small territory . parsing: nsubj_PRON_PronType ROOT_VERB_VerbForm_Tense_Number_Person det_DET amod_ADJ_Degree compound_NOUN_Number dobj_NOUN_Number prep_ADP poss_ADJ_PronType_Poss amod_ADJ_Degree pobj_NOUN_Number .	It has a high - density population because of its small territory ."

'It has a high density population because its small territory . parsing: nsubj_PRON_PronType ROOT_VERB_VerbForm_Tense_Number_Person det_DET amod_ADJ_Degree compound_NOUN_Number dobj_NOUN_Number prep_ADP poss_ADJ_PronType_Poss amod_ADJ_Degree pobj_NOUN_Number .\tIt has a high - density population because of its small territory .'

In [25]:
doc = nlp(sent)

In [4]:
nlp.vocab.morphology.tag_map

{'$': {74: 99},
 "''": {74: 97, 'PunctSide_fin': True, 'PunctType_quot': True},
 ',': {74: 97, 'PunctType_comm': True},
 '-LRB-': {74: 97, 'PunctSide_ini': True, 'PunctType_brck': True},
 '-RRB-': {74: 97, 'PunctSide_fin': True, 'PunctType_brck': True},
 '.': {74: 97, 'PunctType_peri': True},
 ':': {74: 97},
 'ADD': {74: 101},
 'AFX': {74: 84, 'Hyph_yes': True},
 'CC': {74: 89, 'ConjType_comp': True},
 'CD': {74: 93, 'NumType_card': True},
 'DT': {74: 90},
 'EX': {74: 95, 'AdvType_ex': True},
 'FW': {74: 101, 'Foreign_yes': True},
 'HYPH': {74: 97, 'PunctType_dash': True},
 'IN': {74: 85},
 'JJ': {74: 84, 'Degree_pos': True},
 'JJR': {74: 84, 'Degree_comp': True},
 'JJS': {74: 84, 'Degree_sup': True},
 'LS': {74: 101, 'NumType_ord': True},
 'MD': {74: 100, 'VerbType_mod': True},
 'NFP': {74: 97},
 'NN': {74: 92, 'Number_sing': True},
 'NNP': {74: 96, 'NounType_prop': True, 'Number_sing': True},
 'NNPS': {74: 96, 'NounType_prop': True, 'Number_plur': True},
 'NNS': {74: 92, 'Number_plur

In [ ]:
doc = nlp(sent)

In [16]:
with open("./jfleg/test/test.src") as fl:
    inputs = fl.readlines()
with open("./jfleg/test/test.ref0") as fl:
    ref0 = fl.readlines()
with open("./jfleg/test/test.ref1") as fl:
    ref1 = fl.readlines()
with open("./jfleg/test/test.ref2") as fl:
    ref2 = fl.readlines()
with open("./jfleg/test/test.ref3") as fl:
    ref3 = fl.readlines()

In [17]:
jfleg = pd.DataFrame({"inputs": inputs, "ref0": ref0, "ref1": ref1, "ref2":ref2, "ref3": ref3})

In [4]:
for col in jfleg.columns:
    jfleg[col] = jfleg[col].map(lambda x: x.replace('\n', ''))

In [5]:
jfleg["inputs"][0]

'New and new technology has been introduced to the society .'

In [6]:
jfleg.to_csv("./heptabot-train-data/jfleg-test.csv")

In [2]:
import torch 
import transformers 
if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")
    

In [21]:
tokenizer = transformers.T5TokenizerFast.from_pretrained("t5-base").save_pretrained(path)

In [3]:
path = "F:/downloads/storage/final_models/final_models/T5-small_distilled/"
#model = transformers.T5ForConditionalGeneration.from_pretrained(path)
tokenizer = T5TokenizerFast.from_pretrained(path)
model.eval();

NameError: name 'T5TokenizerFast' is not defined

In [18]:
path = "F:/downloads/storage/final_models/final_models/T5-base-quantized/model.checkpoint"
teacher = transformers.T5ForConditionalGeneration.from_pretrained("google/t5-v1_1-base")
#model = transformers.T5ForConditionalGeneration.from_pretrained(path)
#tokenizer = transformers.T5TokenizerFast.from_pretrained(path)
model.eval();
teacher.load_state_dict(torch.load(path))

NameError: name 'model' is not defined

In [19]:
path = "F:/downloads/storage/final_models/final_models/T5-base-quantized/"
teacher.save_pretrained(path)

In [20]:
tokenizer.save_pretrained(path)

AttributeError: 'tuple' object has no attribute 'save_pretrained'

In [5]:
tokenizer = transformers.T5TokenizerFast.from_pretrained(path)

In [6]:
import re
def normalize_text(text):
    """Remove quotes from a TensorFlow string."""
    text = re.sub("'(.*)'", r"\1", text)
    return text

In [32]:
scorr = pd.read_csv("F:/downloads/storage/heptabot-train-data/correction-eval.tsv", sep="\t", header=None)

In [33]:
scorr[0][0], scorr[1][0]

('Armageddon is my favourite science fiction movie. The plot is about how to survive in the bad situation. It presented the cooperation from everybody such as new technology, joining between american and russian astronauts and the private people who had great experience in digging. I felt of the bravehearts who make a sacrifice. The soundtrack was pretty good. I felt sad when the character played by Bruce Willis called his daughter on the earth, prior to exploding a main meteor. Armageddon was directed by Michael Bay.',
 'Armageddon is my favourite science fiction movie. The plot is about how to survive in a bad situation. It presented the cooperation from everybody such as new technology, cooperation between American and Russian astronauts and the private people who had great experience in digging. I felt the bravehearts who make a sacrifice. The soundtrack was pretty good. I felt sad when the character played by Bruce Willis called his daughter on the earth, prior to exploding a main

In [26]:
scorr.head()

,0,1
0,In my contry the nature is very beatiful. Ther...,In my country the nature is very beautiful. Th...
1,The light red dress is 90 $. <br> The orange b...,The light red dress is $90. <br> The orange bi...
2,my neighborhood is big and busy. there's an me...,My neighborhood is big and busy. There's an Me...
3,"Dear Sir or Madam, <br> I am writing to compla...","Dear Sir or Madam, <br> I am writing to compla..."
4,"Hi Brigit, my name 's Priscilla. I'm from Braz...","Hi Brigit, my name 's Priscilla. I'm from Braz..."


In [34]:
def inference(input, task="correction"):
    input = tokenizer.encode_plus(f"{task}: "+input, return_tensors="pt",
                                                     truncation=True, padding='max_length')
    out = model.generate(input_ids=input["input_ids"],
                     attention_mask=input["attention_mask"],
                     max_length=256,
                                    )
    return tokenizer.decode(out[0], skip_special_tokens=True)

In [35]:
%%time
inference(scorr[0][0])

Wall time: 3.18 s


'Armageddon is my favourite science fiction movie. The plot is about how to survive in the bad situation. It presented the cooperation from everybody such as new technology, joining between American and Russian astronauts and the private people who had great experience in digging. I felt of the bravehearts who make a sacrifice. The soundtrack was pretty good. I felt sad when the character played by Bruce Willis called his daughter on the earth, prior to exploding a main meteor. Armageddon was directed by Michael Bay.'

In [ ]:
teacher.generate(
                                        input_ids=ids,
                                        attention_mask=mask,
                                        max_length=256,
                                    )

In [4]:
model.to(device);

In [5]:
teacher = transformers.T5ForConditionalGeneration.from_pretrained("google/t5-v1_1-base")
path = "/home/nasorokin11/modelsbasebestlow2/model-64844.checkpoint"
teacher.load_state_dict(torch.load(path))
teacher.to(device);
teacher.eval();

In [12]:
tokenizer = T5TokenizerFast.from_pretrained('/home/nasorokin11/T5small')
corr_dataset = T5Dataset("./heptabot-train-data/correction-eval.tsv", tokenizer, "correction:", target=1, source=0), [i for i in range(30)]
corr_dataloader = DataLoader(corr_dataset, batch_size=1, num_workers=4
                                 )

In [63]:
from utils import *

In [66]:
scores = 0
for batch in tqdm(corr_dataloader, "", len(corr_dataloader)):
    y = batch['target_ids'].to(device, dtype=torch.long)
    ids = batch['source_ids'].to(device, dtype=torch.long)
    mask = batch['source_mask'].to(device, dtype=torch.long)
    generated_ids = teacher.generate(
                                        input_ids=ids,
                                        attention_mask=mask,
                                        max_length=256,
                                    )
    preds, target, score = get_bleu(generated_ids, y)
    scores += score
    

 75%|███████▌  | 48/64 [06:19<02:06,  7.90s/it]


KeyboardInterrupt: 

In [ ]:
scores/len(corr_dataloader)

In [6]:
model_int8 = torch.quantization.quantize_dynamic(
    teacher.to("cpu"),  # the original model
    {torch.nn.Linear: torch.quantization.default_dynamic_qconfig},  # a set of layers to dynamically quantize
    dtype=torch.qint8)


In [7]:
path = os.path.join("/home/nasorokin11/final_models/T5-base-quantized", "model.checkpoint")
torch.save(model_int8.state_dict(), path)

In [20]:

tokenizer = T5TokenizerFast.from_pretrained('/home/nasorokin11/T5small')
jfleg_dataset = JflegDataset("./heptabot-train-data/jfleg-test.csv", tokenizer, "jfleg:", target=1, source=0)
jfleg_dataloader = DataLoader(jfleg_dataset, batch_size=128, num_workers=4
                                 )

In [18]:
def get_corpus_gleu_jfleg(pred, target, max_length=512):
    bleu = 0
    for i in range(pred.shape[0]):
        p = (tokenizer.decode(pred[i], skip_special_tokens=True)).split(" ")
        t = [(tokenizer.decode(target[i][j][0], skip_special_tokens=True)).split(" ") for j in range(len(target[i]))]
        bleu += sentence_gleu(t, p)
    return p, t, bleu/pred.shape[0]

In [12]:
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedGeluDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
              (wo):

In [21]:
scores = 0
for batch in tqdm(jfleg_dataloader, "", len(jfleg_dataloader)):
    ref_0 = batch['ref_0'].tolist()
    ref_1 = batch['ref_1'].tolist()
    ref_2 = batch['ref_2'].tolist()
    ref_3 = batch['ref_3'].tolist()
    ids = batch['source_ids'].to(device, dtype=torch.long)
    mask = batch['source_mask'].to(device, dtype=torch.long)
    generated_ids = model.generate(
                                        input_ids=ids,
                                        attention_mask=mask,
                                        max_length=256,
                                    )
    y = [[ref_0[i], ref_1[i], ref_2[i], ref_3[i]] for i in range(len(ref_0))]
    preds, target, score = get_corpus_gleu_jfleg(generated_ids, y)
    scores += score
    
    

100%|██████████| 6/6 [00:09<00:00,  1.52s/it]


In [22]:
scores/6

0.7796169800979033

In [34]:
tokenizer.decode(generated_ids[1], skip_special_tokens=True)

'One possible outcome is that an environmentally-induced reduction in motorization levels in the richer countries will outweigh any rise in motorization levels in the poorer countries.'

In [33]:
tokenizer.decode(ids[1], skip_special_tokens=True)

'jfleg: One possible outcome is that an environmentally-induced reduction in motorization levels in the richer countries will outweigh any rise in motorization levels in the poorer countries.'

In [36]:
tokenizer.decode(ref_0[1][0], skip_special_tokens=True)

'One possible outcome is that an environmentally-induced reduction in motorization levels in richer countries will outweigh any rise in motorization levels in poorer countries.'